# Part 2

In part 2 we remove the `worry_level //= 3` and increase the number of rounds from 20 to 10000. This makes the worry level numbers unusably large. So how do we "find another way to keep your worry levels manageable"?

Looking at the input, the test is always divisibility by a prime. The first thing that's apparent is that worry levels of 190,000 and 19 are both divisible by 19. But some operations are multiplication and some are addition. An addition is actually the harder of the two here. If the operation is `old * 13` then it has the same effect on a worry level of 190,000 or 19: the result of both still divisible by 19 and also will be divisible by 13. However if the operation is `old + 1` then item with a worry level of 19 becomes 20 and the 190,000 becomes 190,001. The factors of 190,000 are 7 and 27,143. So a strategy of reducing worry level using division would fail here.

Maybe the worry level of all items could be reduced by some common factor... This would still fail in the face of addition operations.



## Alternate number representations

An integer $N$ is the product of a set of factors

$N = \prod \limits _{i=1} ^{i=S} F_i$

If $N$ is prime then $S$ is 2 and the set of $F$ is just $\{1,N\}$. Non-prime numbers are also called composite. 

Let's look at a few easy examples.

$12 = 3 \times 4$

$12 = 2 \times 2 \times 3$

$1900 = 2 \times 5 \times 10 \times 19$

$1901 = 1 \times 1901$

We're looking for a representation of large numbers which makes it easy to:

1. multiply by one of a small number of factors

The multiplication operations in the input are either a prime number or the number itself.

2. add one of a small list of numbers

There are only a few constants which can be added, all under 10.

3. test for one of a small number of factors

The divisibility test values are all primes.

### Representing an integer as a list of factors

Storing a list like [2, 5, 10, 19] instead of the number 1900 is an option. How does it look?

1. It makes multiplication easy: just add another factor to the list. 

2. It makes addition much harder. Adding $1900 + 1$ makes a prime. Now the entire list of factors needs to be multiplied to create an impossibly large number and re-factored.

3. It might make testing for a prime factor somewhat easier - looking for the factor in the list will find the prime. The number 10 isn't prime but as long as each prime factor is present at least once then it can be found by searching the list.

### Sums and products

Take this representation of $N$, where $A$ and $C$ are prime.

$N = A \times B + C \times D$

If $N$ divisible is by some other integer $E$ then:

$E = A$ or $B$ is divisible by $E$

and

$E = C$ or $D$ is divisible by $E$

This leads to an improvement on the previous representation. Instead of storing a list of factors, store a list of lists of factors. The value is the sum of the products of the inner lists.

$1900 = 2 \times 5 \times 10 \times 19$

$1901 = 1 + 2 \times 5 \times 10 \times 19$

Something like

$N = \sum \limits _{j=1} ^{j=P} \prod \limits _{i=1} ^{i=S_j} F_j,i$

How do the required operations look for this representation?

1. Multiplication: to multiply by $F_k$, find $F_k$ in *each* list. If it's present then increment another number in that list. If it's not present then append it.

2. Addition: factor the addend, create a list of the factors, append that list to the outer list.

3. Test for a factor by checking that the factor is in *all* inner lists.

### Aiming for equivalent operations

Ideally all operations would be simple, for example:

1. Multiplication: increment the count of the multiplicand in all lists

2. Addition: start a new list

3. Divisibility: scan all lists for the factor

There are a fixed set of factors for the divisibility test (after reading the input). There is another fixed set of factors for multiplication operations. The sets can overlap. Suppose there are 7 factors to make a simple equation without loss of generality:

$N = M_1 \times F_1 + M_2 \times F_2 + M_3 \times F_3 + M_4 \times F_4 + M_5 \times F_5 + M_6 \times F_6 + M_7 \times F_7 + R$

This suggests a representation storing pairs of factor and multiplicand with a remainder. Store $N$ as a list of these pairs and the sum of unfactored remaining addends $R$. Here's what the operations would look like:

1. Multiplication by $E$:

$N \times E = E \times ( M_1 \times F_1 + M_2 \times F_2 + M_3 \times F_3 + M_4 \times F_4 + M_5 \times F_5 + M_6 \times F_6 + M_7 \times F_7 + R )$

we know that $E = F_s$ for some $s$ in the range 1 to 7. Suppose $E = F_3$.

$N \times E = M_1 \times F_1 \times F_3 + M_2 \times F_2 \times F_3 + (M_3 + 1) \times F_3 + M_4 \times F_4 \times F_3 + M_5 \times F_5 \times F_3 + M_6 \times F_6 \times F_3 + M_7 \times F_7 \times F_3 + R \times F_3 )$

2. Addition: add to $R$. If $R$ is larger than $M_i / 2$ then factor $R$. Only pull out factors from the set of $F$. Add one to $M_i$ for all factors pulled out.

3. Divisibility: if $F_i > 0$ and $R$ is divisible then it's divisible.

Up to here all representations keep everything needed to actually compute $N$. The value of $N$ isn't needed for the problem at hand. Only the divisibility of $N$ is needed. Specifically divisibility by the fixed set of factors in $F$. Understanding divisiblity doesn't require the exact values of each $M_i$. That is to say, $F_i \times M_i$ has the same effect on divisbility whether $M_i$ is 1 or 1,000,001. So define an operation $D$ that reduces each $M_i$ to 1 if it has a value greater than 0. Then the multiplication example above can be continued:

$D(N \times E) = \hat{M}_1 \times F_1 + \hat{M}_2 \times F_2 + 1 \times F_3 + \hat{M}_4 \times F_4 + \hat{M}_5 \times F_5 + \hat{M}_6 \times F_6 + \hat{M}_7 \times F_7 + R$

Where

* $\hat{M}_i = \begin{cases} 1 & M_i > 0 \\ 0 & M_i = 0 \end{cases}$
* $\hat{M}_3 = 1$ because in this example $N$ was multiplied by $F_3$
* $R \times F_3$ becomes $R$ since factors from the set of $F$ are pulled out and added to $M$


In [ ]:
from dataclasses import dataclass

import monkey

In [ ]:
def business():
  monkeys = monkey.from_file('input.txt')
  def throw_item(m: monkey.MonkeyId, i: monkey.WorryItem):
    monkeys[m].catch(i)
  for round in range(0, 10_000):
    print(f'Round {round + 1}')
    for id, m in monkeys.items():
      # TODO: Reimplement according to part 2 logic above.
      m.take_turn(throw_item, post_inpect_worry_reduction=False)
  counts = sorted([m.get_inspection_count() for m in monkeys.values()])
  print(counts[-1] * counts[-2])

business()